In [ ]:
!git clone https://github.com/arife-16/ASD-project.git /content/ASD-project
!python -m pip install -q -U pip
!python -m pip install -q -r /content/ASD-project/requirements.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os, sys, json, numpy as np, pandas as pd
sys.path.append('/content/ASD-project')
DRIVE_ROOT = '/content/drive/MyDrive/ASD'
PHENO_CSV = os.path.join(DRIVE_ROOT, 'phenotype.csv')
NIFTI_DIR = os.path.join(DRIVE_ROOT, 'nifti')
ATLAS_PATH = os.path.join(DRIVE_ROOT, 'atlas', 'atlas.nii.gz')
TSV_DIR = os.path.join(DRIVE_ROOT, 'confounds_tsv')
WM_MASK = os.path.join(DRIVE_ROOT, 'masks', 'wm.nii.gz')
CSF_MASK = os.path.join(DRIVE_ROOT, 'masks', 'csf.nii.gz')
OUTPUT_DIR = os.path.join(DRIVE_ROOT, 'outputs')
ROI_TS_DIR = os.path.join(OUTPUT_DIR, '_roi_ts')
FEATURES_DIR = os.path.join(OUTPUT_DIR, 'features')
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(ROI_TS_DIR, exist_ok=True)
os.makedirs(FEATURES_DIR, exist_ok=True)


In [ ]:
from asd_pipeline.atlas import extract_roi_timeseries
from asd_pipeline.confounds import build_confounds
from asd_pipeline.preprocess import regress_confounds
pheno = pd.read_csv(PHENO_CSV)
subject_ids = pheno['SUB_ID'].astype(str).tolist()
for sid in subject_ids:
    nii = os.path.join(NIFTI_DIR, f'{sid}.nii.gz')
    if not os.path.exists(nii):
        continue
    ts = extract_roi_timeseries(nii, ATLAS_PATH, atlas_type='labels', tr=2.0, high_pass=0.01, low_pass=0.1)
    tsv = os.path.join(TSV_DIR, f'{sid}_confounds.tsv')
    conf = build_confounds(nii, tsv, WM_MASK, CSF_MASK, 2.0)
    if conf is not None:
        ts = regress_confounds(ts, confounds=conf, tr=2.0, high_pass=0.01, low_pass=0.1)
    np.save(os.path.join(ROI_TS_DIR, f'{sid}.npy'), ts)


In [ ]:
from asd_pipeline.features import build_feature_vector_with_states
X = []
for sid in subject_ids:
    p = os.path.join(ROI_TS_DIR, f'{sid}.npy')
    if not os.path.exists(p):
        continue
    ts = np.load(p)
    vec, _ = build_feature_vector_with_states(ts, tr=2.0, window_size=50, step=10, n_states=5)
    X.append(vec)
X = np.stack(X, axis=0)
np.save(os.path.join(FEATURES_DIR, 'features.npy'), X)


In [ ]:
from asd_pipeline.harmonize import combat_harmonize
covars = pheno[['SITE', 'AGE_AT_SCAN', 'SEX']].copy()
X_h, _ = combat_harmonize(X, covars[['SITE', 'AGE_AT_SCAN', 'SEX']], site_col='SITE', continuous_covars=['AGE_AT_SCAN'], categorical_covars=['SEX'])
np.save(os.path.join(FEATURES_DIR, 'features_harmonized.npy'), X_h)


In [ ]:
from asd_pipeline.normative import personalized_deviation_maps
y = pheno['DX_GROUP'].values.astype(int)
td_mask = y == 2
covars_arr = covars[['AGE_AT_SCAN', 'SEX']].values.astype(float)
dev = personalized_deviation_maps(X_h[td_mask], X_h, covars=covars_arr)
X_dev = dev['feature_z']
np.save(os.path.join(FEATURES_DIR, 'features_deviation.npy'), X_dev)


In [ ]:
from asd_pipeline.model import evaluate_models
groups = pheno['SITE'].values
model_metrics = evaluate_models(X_dev, y, cv_strategy='site_stratified', groups=groups, cv_splits=5)
out_path = os.path.join(OUTPUT_DIR, 'metrics.json')
with open(out_path, 'w') as f:
    json.dump({'models': model_metrics}, f)
print(json.dumps({'models': model_metrics}))
